In [11]:
import numpy as np
import pandas as pd
import itertools
from itertools import combinations, chain
from scipy.special import comb

In [3]:
value_dict = {"2": 2, "3": 3, "4": 4, "5": 5, "6": 6, "7":7, "8": 8, "9": 9, "10": 10, "J": 11, "Q": 12, "K": 13, "A": 14}
suit_dict = {"d": 0, "c": 1, "s": 2, "h": 3}

In [4]:
num = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"]
suit = ["d", "c", "s", "h"]

In [5]:
deck = [n + ' ' +  s for n in num for s in suit]

In [7]:
for idx, card in enumerate(deck):
    deck[idx] = np.array(
    [value_dict[card.split(' ')[0]], suit_dict[card.split(' ')[1]]])

In [8]:
deck_arr = np.array(deck)

In [10]:
def comb_index(n, k):
    count = comb(n, k, exact=True)
    index = np.fromiter(chain.from_iterable(combinations(range(n), k)), 
                        int, count=count*k)
    return index.reshape(-1, k)

In [51]:
%timeit total_idx = comb_index(52, 5)

911 ms ± 15.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [52]:
%timeit hand_combos = deck_arr[total_idx, :]

391 ms ± 19.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [53]:
%timeit other_hand = np.repeat(np.array([[[3,0], [4,0]]]), len(hand_combos), axis=0)

52.6 ms ± 1.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [56]:
# np.concatenate([hand_combos, other_hand], axis=1)

In [50]:
%timeit np.concatenate([hand_combos, other_hand], axis=1)

191 ms ± 2.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [58]:
%timeit hand_score(hand_combos[1, :, :])

132 µs ± 6.18 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [69]:
%timeit np.apply_along_axis(func1d=hand_score_2, arr=hand_combos[:1000, :, :].reshape(-1,10), axis=1)

164 ms ± 3.95 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%timeit parallel_apply_along_axis(func1d=hand_score_2, arr=hand_combos[:100, :, :].reshape(-1,10), axis=1)

In [68]:
%timeit np.array([hand_score(hand_combos[i, :, :]) for i in range(1000)])

134 ms ± 2.94 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
import timeit

start = timeit.default_timer()
hand_scores = parallel_apply_along_axis(func1d=hand_score,
    arr=hand_combos.reshape(-1, 10),
    axis=1)
end = timeit.default_timer()
print(end - start)

In [62]:
def hand_score(hand):
        
    hand_values = np.sort(hand[:, 0])[::-1]#np.sort(hand[[0,2,4,6,8]])[::-1]
    hand_suits = hand[:, 1]#np.sort(hand[[1,3,5,7,9]])
    
    return assign_rank(hand_suits, hand_values)

def hand_score_2(hand):
        
    hand_values = np.sort(hand[[0,2,4,6,8]])[::-1]
    hand_suits = np.sort(hand[[1,3,5,7,9]])
    
    return assign_rank(hand_suits, hand_values)
    

def suit_check(hand_suits):
    return np.all(hand_suits[0] == hand_suits)

def straight_check(hand_values):
    return np.all(hand_values == np.arange(hand_values[0], hand_values[0] - 5, -1)) | \
           np.all(hand_values == np.array([14,5,4,3,2]))

def assign_rank(hand_suits, hand_values):
    
    is_suited = suit_check(hand_suits)
    is_straight = straight_check(hand_values)
    
    if np.all(hand_values == np.array([2,3,4,5,14])):
        hand_values = np.array([1,2,3,4,5])
    
    if is_suited and is_straight and hand_values[0] == 10:
        rank = np.concatenate([np.array([10]),  hand_values])
    elif is_suited and is_straight:
        rank = np.concatenate([np.array([9]),  hand_values])
    elif is_suited:
        rank = np.concatenate([np.array([6]),  hand_values])
    elif is_straight:
        rank = np.concatenate([np.array([5]),  hand_values])
    else:
        val, count = np.unique(hand_values, return_counts=True)
        rank = np.concatenate(
                [np.array([rank_dict((max(count), min(count), len(count)))]),
                 val[count.argsort()]])
        
#     print(rank)
    return assign_hand_value(rank)

def assign_hand_value(hand_array):
        return int(np.sum(np.exp2(np.arange(20, 20 - len(hand_array) * 4, -4)) * hand_array))
    
def rank_dict(max_min_len):
        rank_params_dict = {(4, 1, 2): 8, (3, 2, 2): 7, (3, 1, 3): 4, (2, 1, 3): 3, (2, 1, 4): 2, (1, 1, 5): 1}
        return rank_params_dict[max_min_len]
    
    

In [63]:
import multiprocessing
def parallel_apply_along_axis(func1d, axis, arr, *args, **kwargs):
    """
    Like numpy.apply_along_axis(), but takes advantage of multiple
    cores.
    """        
    # Effective axis where apply_along_axis() will be applied by each
    # worker (any non-zero axis number would work, so as to allow the use
    # of `np.array_split()`, which is only done on axis 0):
    effective_axis = 1 if axis == 0 else axis
    if effective_axis != axis:
        arr = arr.swapaxes(axis, effective_axis)

    # Chunks for the mapping (only a few chunks):
    chunks = [(func1d, effective_axis, sub_arr, args, kwargs)
              for sub_arr in np.array_split(arr, multiprocessing.cpu_count())]

    pool = multiprocessing.Pool()
    individual_results = pool.map(unpacking_apply_along_axis, chunks)
    # Freeing the workers:
    pool.close()
    pool.join()

    return np.concatenate(individual_results)

def unpacking_apply_along_axis(allargs):
    """
    Like numpy.apply_along_axis(), but with arguments in a tuple
    instead.

    This function is useful with multiprocessing.Pool().map(): (1)
    map() only handles functions that take a single argument, and (2)
    this function can generally be imported from a module, as required
    by map().
    """
    (func1d, axis, arr, args, kwargs) = all_args